
# Laboratorio 06--> Hunting with Data Science



Universidad del Valle de Guatemala\
Security Data Science\
Luis Pedro Cuéllar - 18220


### PARTE I

In [2]:
import re
import json
import pandas as pd
import numpy as np

In [3]:
## Cargar archivo json a una lista
data = []
file = open('data/large_eve.json', 'r')

count = 0

for line in file:
    data.append(line.strip())

In [4]:
## Mostrar cantidad de registros en la lista
len(data)

746909

In [5]:
## Filtramos la lista original para incluír solo a aquellos registros que tengan la llave "dns"
dns_data = []
    
for i in data:
    jsn = json.loads(i)
    
    if 'dns' in jsn:
        dns_data.append(jsn)

In [6]:
## Mostramos la cantidad de registros que tiene la nueva lista filtrada
len(dns_data)

15749

In [7]:
## Mostramos dos registros al azar
dns_data[0]

{'timestamp': '2017-07-22T17:33:16.661646-0500',
 'flow_id': 1327836194150542,
 'pcap_cnt': 22269,
 'event_type': 'dns',
 'vlan': 110,
 'src_ip': '2001:0dbb:0c18:0011:0260:6eff:fe30:0863',
 'src_port': 59680,
 'dest_ip': '2001:0500:0001:0000:0000:0000:803f:0235',
 'dest_port': 53,
 'proto': 'UDP',
 'dns': {'type': 'query',
  'id': 15529,
  'rrname': 'api.wunderground.com',
  'rrtype': 'A',
  'tx_id': 0}}

In [8]:
dns_data[-1]

{'timestamp': '2017-07-22T19:47:38.146953-0500',
 'flow_id': 2199315651640391,
 'pcap_cnt': 4452918,
 'event_type': 'dns',
 'vlan': 130,
 'src_ip': '192.168.207.4',
 'src_port': 53,
 'dest_ip': '192.168.203.67',
 'dest_port': 50975,
 'proto': 'UDP',
 'dns': {'type': 'answer',
  'id': 35892,
  'rcode': 'NXDOMAIN',
  'rrname': '<root>',
  'rrtype': 'SOA',
  'ttl': 20864}}

In [9]:
## Normalizamos la lista de la lista filtrada para poder convertirla en un dataset
df = pd.json_normalize(dns_data)
df.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,2017-07-22T17:33:27.380146-0500,578544790391795,54520,dns,150,192.168.207.4,53,192.168.205.170,31393,UDP,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


In [10]:
## Filtramos el df para obtener solo los registros cuya ip destino y el dominio sean iguales
type_a = df[df['dns.rrtype'] == "A"]

In [11]:
len(type_a)

2849

In [12]:
## Filtramos los dominios únicos
unique_domains_df = type_a.drop_duplicates(subset=['dns.rrname'])

In [13]:
unique_domains_df.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN


In [14]:
unique_domains_df['dns.rrname'].unique()

array(['api.wunderground.com', 'stork79.dropbox.com',
       'hpca-tier2.office.aol.com.ad.aol.aoltw.net',
       'safebrowsing.clients.google.com.home', 'fxfeeds.mozilla.com',
       'www.metasploit.com', 'aolmtcmxm03.office.aol.com',
       'aolmtcmxm02.office.aol.com.ad.aol.aoltw.net',
       'aolmtcmxm02.office.aol.com', 'hpca-tier2.office.aol.com',
       'aolmtcmxm03.office.aol.com.ad.aol.aoltw.net',
       'aolmtcmxm04.office.aol.com', 'safebrowsing.clients.google.com',
       'wpad.home', 'safebrowsing.clients.google.com.stayonline.net',
       'aolmtcmxm04.office.aol.com.ad.aol.aoltw.net',
       'AOLDTCMA04.ad.aol.aoltw.net.office.aol.com',
       'AOLDTCMA04.office.aol.com', '192.168.22.110phpmyadmin',
       'secure.informaction.com', 'secure.informaction.com.localdomain',
       'safebrowsing.clients.google.com.localdomain', 'ueip.vmware.com',
       'en-us.fxfeeds.mozilla.com',
       '192.168.22.110phpmyadmin.localdomain', 'time.windows.com',
       'softwareupdate.vmwar

In [15]:
!pip install tldextract

In [16]:
## Funciones para obtener el TLD de un dominio

import tldextract

def check_for_ip(value):
    has_ip = re.match("^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", value)
    
    if has_ip:
        return True

    else:
        return False
    
def get_tld(domain):
    has_ip = check_for_ip(domain)
    
    if has_ip:
        return domain
    
    else:
        try:
            ext = tldextract.extract(domain)
            
            if ext.suffix:
                return '{}.{}'.format(ext.domain, ext.suffix)
            
            return domain
        except:
            return ' '

In [17]:
## Obtenemos el TLD de dominio y lo asignamos a una nueva columna

unique_domains_df['tld'] = unique_domains_df['dns.rrname'].apply(get_tld)

/var/folders/kh/vcfbrgzs4d34zw7_9vlbmc780000gn/T/ipykernel_2657/4171429463.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_domains_df['tld'] = unique_domains_df['dns.rrname'].apply(get_tld)


In [18]:
unique_domains_df.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,tld
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN,wunderground.com
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN,dropbox.com
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN,aoltw.net
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN,safebrowsing.clients.google.com.home
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN,mozilla.com


In [19]:
unique_domains_df['tld'].unique()

array(['wunderground.com', 'dropbox.com', 'aoltw.net',
       'safebrowsing.clients.google.com.home', 'mozilla.com',
       'metasploit.com', 'aol.com', 'google.com', 'wpad.home',
       'stayonline.net', '192.168.22.110phpmyadmin', 'informaction.com',
       'secure.informaction.com.localdomain',
       'safebrowsing.clients.google.com.localdomain', 'vmware.com',
       '192.168.22.110phpmyadmin.localdomain', 'windows.com',
       'ntkrnlpa.info', 'portswigger.net', 'offensive-security.com',
       'stopbadware.org', 'arrancar.org', 'sql-ledger.org',
       'backtrack-linux.org', 'theanime.cn',
       'safebrowsing.clients.google.com.hackerlabs.vpn', 'phpmyadmin.net',
       'microsoft.com', 'secure.informaction.com.home', 'comcast.net',
       'ubuntu.com', 'alexa.com', 'postgresql.org', 'sourceforge.net',
       'freepbx.org', 'secure.informaction.com.hackerlabs.vpn',
       'bigflickrfeed.com', 'gnu.org', 'wpad',
       'safebrowsing.clients.google.com.lan', '"192.168.206.56"',
   

### PARTE II

In [20]:
## Eliminamos la columna que no tiene dominio ya que dará problemas en algunas funciones para el modelo

unique_domains_df[unique_domains_df['tld'].str.len() == 0]

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,dns.type,dns.id,dns.rrname,dns.rrtype,dns.tx_id,dns.rcode,dns.ttl,dns.rdata,tld
6904,2017-07-22T18:38:22.596962-0500,43896779185122,596043,dns,110,192.168.201.72,60152,192.168.207.4,53,UDP,query,24093,,A,0.0,NaN,NaN,NaN,


In [21]:
unique_domains_df = unique_domains_df[unique_domains_df['tld'].str.len() != 0]

In [22]:
## Como parte del modelo de clasificación, tenemos que generar todas las features en base al tld

from urllib.parse import urlparse

## Función f1 -> longitud de la URL completa
unique_domains_df['len_url'] = unique_domains_df.apply(lambda row: len(row['tld']), axis = 1)

## Función f4 -> '.'
unique_domains_df['qty_points'] = unique_domains_df.apply(lambda row: row['tld'].count('.'), axis = 1)

## Función f5 -> '-'
unique_domains_df['qty_hyphen'] = unique_domains_df.apply(lambda row: row['tld'].count('-'), axis = 1)

## Función f26 -> ratio of numbers in full URL
unique_domains_df['numbers_url_ratio'] = unique_domains_df.apply(lambda row: len((re.sub("[^0-9]", "", row['tld'])))/len(row['tld']), axis = 1)

In [39]:
## funciones del archivo "caracteristicasDerivadas.py" en canvas

def posicionPrimerDigito( s ):
    for i, c in enumerate(s):
        if c.isdigit():
            return i + 1
    return 0
def proporcionVocalesConsonantes (x):
    x = x.lower()
    patron_vocales = re.compile('([aeiou])')
    patron_consonantes = re.compile('([b-df-hj-np-tv-z])')
    vocales = re.findall(patron_vocales, x)
    consonantes = re.findall(patron_consonantes, x)
    try:
        proporcion = len(vocales) / len(consonantes)
    except: # Division por cero
        proporcion = 0  
    return proporcion

def H_entropy (x):
    # Entropia de Shannon
    prob = [ float(x.count(c)) / len(x) for c in dict.fromkeys(list(x)) ] 
    H = - sum([ p * np.log2(p) for p in prob ]) 
    return H

In [40]:
unique_domains_df['entropy'] = unique_domains_df['tld'].apply(H_entropy)

unique_domains_df['vocals_consonant_ratio'] = unique_domains_df['tld'].apply(proporcionVocalesConsonantes)

unique_domains_df['first_digit_position'] = unique_domains_df['tld'].apply(posicionPrimerDigito)

In [41]:
unique_domains_df.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,dns.ttl,dns.rdata,tld,len_url,qty_points,qty_hyphen,numbers_url_ratio,vocals_consonant_ratio,first_digit_position,entropy
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,...,NaN,NaN,wunderground.com,16,1,0,0.0,0.500000,0,3.375000
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,...,NaN,NaN,dropbox.com,11,1,0,0.0,0.428571,0,3.027169
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,...,NaN,NaN,aoltw.net,9,1,0,0.0,0.600000,0,2.947703
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,...,NaN,NaN,safebrowsing.clients.google.com.home,36,4,0,0.0,0.600000,0,3.861052
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,...,NaN,NaN,mozilla.com,11,1,0,0.0,0.666667,0,2.913977


In [42]:
nan_value = float("NaN")
unique_domains_df.replace("", nan_value, inplace=True)

unique_domains_df.dropna(subset = ["tld"], inplace=True)

In [43]:
unique_domains_df.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,dns.ttl,dns.rdata,tld,len_url,qty_points,qty_hyphen,numbers_url_ratio,vocals_consonant_ratio,first_digit_position,entropy
0,2017-07-22T17:33:16.661646-0500,1327836194150542,22269,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680,2001:0500:0001:0000:0000:0000:803f:0235,53,UDP,...,NaN,NaN,wunderground.com,16,1,0,0.0,0.500000,0,3.375000
1,2017-07-22T17:33:24.990320-0500,2022925111925872,54352,dns,110,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051,2001:0500:0003:0000:0000:0000:0000:0042,53,UDP,...,NaN,NaN,dropbox.com,11,1,0,0.0,0.428571,0,3.027169
2,2017-07-22T17:33:27.379891-0500,578544790391795,54519,dns,150,192.168.205.170,31393,192.168.207.4,53,UDP,...,NaN,NaN,aoltw.net,9,1,0,0.0,0.600000,0,2.947703
6,2017-07-22T17:33:38.537426-0500,2167545251640146,55687,dns,180,192.168.198.62,35092,192.168.207.4,53,UDP,...,NaN,NaN,safebrowsing.clients.google.com.home,36,4,0,0.0,0.600000,0,3.861052
9,2017-07-22T17:33:45.672579-0500,541745511744323,56307,dns,120,192.168.202.172,55580,192.168.207.4,53,UDP,...,NaN,NaN,mozilla.com,11,1,0,0.0,0.666667,0,2.913977


In [44]:
## Generamos un nuevo df que va a tener las features que utilizaremos para el modelo

features = pd.DataFrame()

features['longitud'] = unique_domains_df['len_url']
features['digitos'] = unique_domains_df['numbers_url_ratio']
features['entropia'] = unique_domains_df['entropy']
features['proporcionVocalesConsonantes'] = unique_domains_df['vocals_consonant_ratio']
features['posicionPrimerDigito'] = unique_domains_df['first_digit_position']

features.head()

,longitud,digitos,entropia,proporcionVocalesConsonantes,posicionPrimerDigito
0,16,0.0,3.375000,0.500000,0
1,11,0.0,3.027169,0.428571,0
2,9,0.0,2.947703,0.600000,0
6,36,0.0,3.861052,0.600000,0
9,11,0.0,2.913977,0.666667,0


In [45]:
## Importamos el modelo creado en el archivo subido en canvas "DGA.py"

from joblib import load

clf = load('references/clf.joblib')

predictions = clf.predict(features)

In [46]:
predictions

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0])

In [47]:
## Filtramos todos los dominios considerados como DGA

dga_domains = []

for i in predictions:
    if i == 1:
        dga_domains.append(True)
    
    else:
        dga_domains.append(False)


In [48]:
filtered_domains = unique_domains_df[dga_domains]
filtered_domains['tld'].unique()

array(['safebrowsing.clients.google.com.home', '192.168.22.110phpmyadmin',
       'safebrowsing.clients.google.com.localdomain',
       '192.168.22.110phpmyadmin.localdomain', 'ntkrnlpa.info',
       'portswigger.net', 'sql-ledger.org', 'backtrack-linux.org',
       'safebrowsing.clients.google.com.hackerlabs.vpn', 'phpmyadmin.net',
       'postgresql.org', 'secure.informaction.com.hackerlabs.vpn',
       'safebrowsing.clients.google.com.lan', '"192.168.206.56"',
       'windowsupdate.com', '192.168.26-27.0', 'fastserv.com',
       'liquidweb.com', 'team-cymru.org', '192.168.21.1201',
       'msftncsi.com', 'vtlfccmfxlkgifuf.com', '1922.168.22.254',
       '192.168.21-28.0', 'ejfodfmfxlkgifuf.xyz', '192.168.22.201:',
       '192.168.22.201:.stayonline.net'], dtype=object)

### PARTE III

In [49]:
## Buscamos si los dominios en la lista, se encuentran en la lista de Cisco Umbrella

from flare.tools.umbrella import Umbrella

umbrellaCisco = Umbrella(limit=1000000)
sus_domains = filtered_domains['tld'].unique()
umbrella_registered = []
umbrella_not_registered = []

for i in sus_domains:
    if umbrellaCisco.domain_in_umbrella(i):
        umbrella_registered.append(i)
    
    else:
        umbrella_not_registered.append(i)

[+] Updated Umbrella Top 1 Million list...


In [50]:
umbrella_registered

['portswigger.net',
 'phpmyadmin.net',
 'postgresql.org',
 'windowsupdate.com',
 'fastserv.com',
 'liquidweb.com',
 'team-cymru.org',
 'msftncsi.com']

In [51]:
umbrella_not_registered

['safebrowsing.clients.google.com.home',
 '192.168.22.110phpmyadmin',
 'safebrowsing.clients.google.com.localdomain',
 '192.168.22.110phpmyadmin.localdomain',
 'ntkrnlpa.info',
 'sql-ledger.org',
 'backtrack-linux.org',
 'safebrowsing.clients.google.com.hackerlabs.vpn',
 'secure.informaction.com.hackerlabs.vpn',
 'safebrowsing.clients.google.com.lan',
 '"192.168.206.56"',
 '192.168.26-27.0',
 '192.168.21.1201',
 'vtlfccmfxlkgifuf.com',
 '1922.168.22.254',
 '192.168.21-28.0',
 'ejfodfmfxlkgifuf.xyz',
 '192.168.22.201:',
 '192.168.22.201:.stayonline.net']

In [52]:
## Creación de la función para obtener la fecha de creación del dominio

import whois

for i in umbrella_not_registered:
    date=whois.whois(i).creation_date
    print(i, date)

safebrowsing.clients.google.com.home None
192.168.22.110phpmyadmin None
safebrowsing.clients.google.com.localdomain None
192.168.22.110phpmyadmin.localdomain None
ntkrnlpa.info 2021-09-20 18:01:53
sql-ledger.org 2000-09-08 16:25:09
backtrack-linux.org [datetime.datetime(2009, 4, 29, 0, 26, 42), datetime.datetime(2009, 4, 28, 22, 26, 42)]
safebrowsing.clients.google.com.hackerlabs.vpn None
secure.informaction.com.hackerlabs.vpn None
safebrowsing.clients.google.com.lan None
"192.168.206.56" None
192.168.26-27.0 None
192.168.21.1201 None


PywhoisError: No match for "VTLFCCMFXLKGIFUF.COM".
>>> Last update of whois database: 2022-05-03T02:57:20Z <<<

NOTICE: The expiration date displayed in this record is the date the
registrar's sponsorship of the domain name registration in the registry is
currently set to expire. This date does not necessarily reflect the expiration
date of the domain name registrant's agreement with the sponsoring
registrar.  Users may consult the sponsoring registrar's Whois database to
view the registrar's reported date of expiration for this registration.

TERMS OF USE: You are not authorized to access or query our Whois
database through the use of electronic processes that are high-volume and
automated except as reasonably necessary to register domain names or
modify existing registrations; the Data in VeriSign Global Registry
Services' ("VeriSign") Whois database is provided by VeriSign for
information purposes only, and to assist persons in obtaining information
about or related 